# 第6章: 単語ベクトル

単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して、以下の処理を行うプログラムを作成せよ。

In [1]:
!pwd

/content


## 50. 単語ベクトルの読み込みと表示

Google Newsデータセット（約1,000億単語）での[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）をダウンロードし、"United States"の単語ベクトルを表示せよ。ただし、"United States"は内部的には"United_States"と表現されていることに注意せよ。

In [1]:
!pip install gensim

In [2]:
import gensim.downloader as api

# Google Newsの学習済み単語ベクトルをダウンロード・ロード
wv = api.load('word2vec-google-news-300')

# "United_States" の単語ベクトルを表示
if "United_States" in wv:
    print(wv["United_States"])
else:
    print("単語 'United_States' はボキャブラリに含まれていません。")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
[-3.61328125e-02 -4.83398438e-02  2.35351562e-01  1.74804688e-01
 -1.46484375e-01 -7.42187500e-02 -1.01562500e-01 -7.71484375e-02
  1.09375000e-01 -5.71289062e-02 -1.48437500e-01 -6.00585938e-02
  1.74804688e-01 -7.71484375e-02  2.58789062e-02 -7.66601562e-02
 -3.80859375e-02  1.35742188e-01  3.75976562e-02 -4.19921875e-02
 -3.56445312e-02  5.34667969e-02  3.68118286e-04 -1.66992188e-01
 -1.17187500e-01  1.41601562e-01 -1.69921875e-01 -6.49414062e-02
 -1.66992188e-01  1.00585938e-01  1.15722656e-01 -2.18750000e-01
 -9.86328125e-02 -2.56347656e-02  1.23046875e-01 -3.54003906e-02
 -1.58203125e-01 -1.60156250e-01  2.94189453e-02  8.15429688e-02
  6.88476562e-02  1.87500000e-01  6.49414062e-02  1.15234375e-01
 -2.27050781e-02  3.32031250e-01 -3.27148438e-02  1.77734375e-01
 -2.08007812e-01  4.54101562e-02 -1.23901367e-02  1.19628906e-01
  7.44628906e-03 -9.03320312e-03  1.14257812e-01  1.69921875e-01
 -2

## 51. 単語の類似度

"United States"と"U.S."のコサイン類似度を計算せよ。

In [3]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Google Newsの学習済み単語ベクトルをロード (すでにロード済みであればスキップ)
try:
    wv
except NameError:
    wv = api.load('word2vec-google-news-300')

# 単語が存在するか確認し、ベクトルを取得
if "United_States" in wv and "U.S." in wv:
    vector_us = wv["United_States"]
    vector_us_short = wv["U.S."]

    # コサイン類似度を計算 (scikit-learnを使用)
    similarity = cosine_similarity([vector_us], [vector_us_short])[0][0]
    print(f'"United States"と"U.S."のコサイン類似度: {similarity}')

else:
    print("指定された単語のいずれか、または両方がボキャブラリに含まれていません。")

"United States"と"U.S."のコサイン類似度: 0.7310774922370911


## 52. 類似度の高い単語10件

"United States"とコサイン類似度が高い10語と、その類似度を出力せよ。

In [4]:
import gensim.downloader as api
from gensim.models import KeyedVectors

# Google Newsの学習済み単語ベクトルをロード (すでにロード済みであればスキップ)
try:
    wv
except NameError:
    wv = api.load('word2vec-google-news-300')

# 単語が存在するか確認
if "United_States" in wv:
    # "United_States" との類似度が高い上位10語を取得
    similar_words = wv.most_similar("United_States", topn=10)

    print('"United States"とコサイン類似度が高い上位10語:')
    for word, similarity in similar_words:
        print(f'{word}: {similarity:.4f}')
else:
    print("単語 'United_States' はボキャブラリに含まれていません。")

"United States"とコサイン類似度が高い上位10語:
Unites_States: 0.7877
Untied_States: 0.7541
United_Sates: 0.7401
U.S.: 0.7311
theUnited_States: 0.6404
America: 0.6178
UnitedStates: 0.6167
Europe: 0.6133
countries: 0.6045
Canada: 0.6019


## 53. 加法構成性によるアナロジー

"Spain"の単語ベクトルから"Madrid"のベクトルを引き、"Athens"のベクトルを足したベクトルを計算し、そのベクトルと類似度の高い10語とその類似度を出力せよ。

In [5]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import numpy as np

# Google Newsの学習済み単語ベクトルをロード (すでにロード済みであればスキップ)
try:
    wv
except NameError:
    wv = api.load('word2vec-google-news-300')

# 関連する単語がボキャブラリに存在するか確認
if "Spain" in wv and "Madrid" in wv and "Athens" in wv:
    # ベクトルの演算: Spain - Madrid + Athens
    vector_result = wv["Spain"] - wv["Madrid"] + wv["Athens"]

    # 結果ベクトルと類似度が高い上位10語を取得
    similar_words = wv.most_similar(positive=[vector_result], topn=10)

    print('"Spain" - "Madrid" + "Athens" のベクトルとコサイン類似度が高い上位10語:')
    for word, similarity in similar_words:
        print(f'{word}: {similarity:.4f}')
else:
    print("指定された単語の一部、または全てがボキャブラリに含まれていません。")

"Spain" - "Madrid" + "Athens" のベクトルとコサイン類似度が高い上位10語:
Athens: 0.7528
Greece: 0.6685
Aristeidis_Grigoriadis: 0.5496
Ioannis_Drymonakos: 0.5361
Greeks: 0.5352
Ioannis_Christou: 0.5330
Hrysopiyi_Devetzi: 0.5088
Iraklion: 0.5059
Greek: 0.5041
Athens_Greece: 0.5034


## 54. アナロジーデータでの実験

[単語アナロジーの評価データ](http://download.tensorflow.org/data/questions-words.txt)をダウンロードし、国と首都に関する事例（`: capital-common-countries`セクション）に対して、vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し、そのベクトルと類似度が最も高い単語と、その類似度を求めよ。求めた単語と類似度は、各事例と一緒に記録せよ。

## 55. アナロジータスクでの正解率

54の実行結果を用い、意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ。

## 56. WordSimilarity-353での評価

[The WordSimilarity-353 Test Collection](http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.html)の評価データをダウンロードし、単語ベクトルにより計算される類似度のランキングと、人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ。

## 57. k-meansクラスタリング

国名に関する単語ベクトルを抽出し、k-meansクラスタリングをクラスタ数k=5として実行せよ。

## 58. Ward法によるクラスタリング

国名に関する単語ベクトルに対し、Ward法による階層型クラスタリングを実行せよ。さらに、クラスタリング結果をデンドログラムとして可視化せよ。

## 59. t-SNEによる可視化

ベクトル空間上の国名に関する単語ベクトルをt-SNEで可視化せよ。